## One_direction_tables_by_routes

### Import libraries

In [1]:
# Step 1: Import libraries
import pandas as pd
import numpy as np
import geopandas as gpd
from sqlalchemy import create_engine
from sqlalchemy import text
from shapely.geometry import LineString, Point
from shapely.ops import unary_union

### Create the connection with the DB (optional)

In [2]:
# Step 1: Create the connection with the DB
engine = create_engine("postgresql://urbaninfo:@cirrus.ita.chalmers.se/se_tuptp")
conn = engine.connect()

In [3]:
# Step 2: Create a new schema when it is necessary
#schema = '''CREATE SCHEMA name_schema'''
#conn.execute(schema)

### Nodes

In [4]:
# ImportData
# Step 1: Import Data
nodes_pt_1_query = text('SELECT * FROM pt_1_mdirections_mstops_mroutes.nodes_pt_1')
nodes_pt_1 = gpd.read_postgis(nodes_pt_1_query, engine, geom_col='geometry')

In [5]:
# Preprocess the data.
# Step 1: Copy the original DataFrame
nodes_pt_2_id = nodes_pt_1.copy()

# Step 2: Create the 'node_id' column by concatenating 'stop_name', 'place_id', and 'route_id' with underscores
nodes_pt_2_id['node_id'] = nodes_pt_2_id['stop_name'].astype(str) + "_" + \
                           nodes_pt_2_id['place_id'].astype(str) + "_" + \
                           nodes_pt_2_id['route_id'].astype(str)

# Step 3: Define aggregation functions for each column
agg_func_node = {
    'route_id': 'first',
    'route_short_name': 'first',
    'stop_id': list,
    'stop_name': 'first',
    'place_id': 'first',
    'mode': 'first',
    'direction_id': list,
    'osmid': list,
    'geometry': lambda g: Point(unary_union(g).centroid.x, unary_union(g).centroid.y)
}

nodes_pt_2_grouped = nodes_pt_2_id.groupby('node_id').agg(agg_func_node).reset_index()

In [6]:
# Filter and organize the data
# Step 4: Select and clean the final columns
nodes_pt_2 = nodes_pt_2_grouped[['node_id', 'stop_name', 'place_id', 'route_id',
                                 'route_short_name', 'mode', 'stop_id',
                                 'direction_id', 'osmid', 'geometry']].dropna().drop_duplicates('node_id')

# Step 5 (optional): Display the head of the DataFrame
nodes_pt_2.head()

,node_id,stop_name,place_id,route_id,route_short_name,mode,stop_id,direction_id,osmid,geometry
0,7:e Villagatan_307955489_9011014200100000,7:e Villagatan,307955489,9011014200100000,1,bus_service,"[9022014082785001, 9022014082785002]","[0, 1]","[87793055.0, 23859327.0]",POINT (378291.977 6399668.440)
1,7:e Villagatan_307955489_9011014200300000,7:e Villagatan,307955489,9011014200300000,3,bus_service,"[9022014082785001, 9022014082785002]","[0, 1]","[87793055.0, 23859327.0]",POINT (378291.977 6399668.440)
2,7:e Villagatan_307955489_9011014200500000,7:e Villagatan,307955489,9011014200500000,5,bus_service,"[9022014082785001, 9022014082785002]","[1, 0]","[87793055.0, 23859327.0]",POINT (378291.977 6399668.440)
3,7:e Villagatan_307955489_9011014200800000,7:e Villagatan,307955489,9011014200800000,8,bus_service,"[9022014082785001, 9022014082785002]","[0, 1]","[87793055.0, 23859327.0]",POINT (378291.977 6399668.440)
4,AGA Vattenfall_307980814_9011014633100000,AGA Vattenfall,307980814,9011014633100000,331,bus_service,"[9022014015161001, 9022014015161002]","[0, 1]","[1021141732.0, 1021141732.0]",POINT (313191.955 6442732.480)


In [7]:
# Step 6 (optional): Set geometry and export to the DB
nodes_pt_2 = gpd.GeoDataFrame(nodes_pt_2, geometry='geometry', crs='3006')
nodes_pt_2.to_postgis('nodes_pt_2', engine, schema='pt_2_odirection_mstops_mroutes', if_exists ='replace')

### Links

In [10]:
# Import data
#Step 1: Import Data
links_pt_1_wide_query = text('SELECT * FROM pt_1_mdirections_mstops_mroutes.links_pt_1_wide')
links_pt_1 = gpd.read_postgis(links_pt_1_wide_query, engine, geom_col='geometry')

In [11]:
# Preprocess the Data
# Step 1: Merge nodes and links in order to get the right combination of data
links_pt_2_source_merge = pd.merge(
    nodes_pt_2[['node_id', 'stop_id', 'stop_name', 'place_id', 'route_id', 'direction_id', 'geometry']],
    links_pt_1 [['route_id', 'route_short_name', 'stop_id_source', 'stop_id_target',
                     'stop_name_source', 'stop_name_target', 'time_distance',
                     'place_id_source', 'place_id_target', 'mode'] + [col for col in links_pt_1.columns if col.startswith('freq_')]],
    left_on=['route_id', 'stop_name', 'place_id'],
    right_on=['route_id', 'stop_name_source', 'place_id_source'],
    how='inner'
)

links_pt_2_target_merge = pd.merge(
    nodes_pt_2[['node_id', 'stop_id', 'route_id', 'stop_name', 'place_id', 'direction_id', 'geometry']],
    links_pt_2_source_merge[['node_id', 'stop_name_source', 'stop_name_target', 'place_id_source', 'place_id_target',
                             'stop_id_source', 'stop_id_target', 'time_distance', 'route_id', 'route_short_name',
                             'mode', 'geometry'] + [col for col in links_pt_2_source_merge.columns if col.startswith('freq_')]],
    left_on=['route_id', 'stop_name', 'place_id'],
    right_on=['route_id', 'stop_name_target', 'place_id_target'],
    how='inner'
).rename(columns={'node_id_x': 'node_id_target', 'node_id_y': 'node_id_source', 'geometry_x': 'geometry_target', 'geometry_y': 'geometry_source'})

# Step 2: Select specific columns and remove duplicates
links_pt_2_l1 = links_pt_2_target_merge[['node_id_source', 'node_id_target', 'time_distance', 'stop_name_source', 'stop_name_target',
                                             'place_id_source', 'place_id_target', 'stop_id_source', 'stop_id_target', 'route_id', 'route_short_name',
                                             'mode', 'direction_id', 'geometry_source', 'geometry_target'] + [col for col in links_pt_2_target_merge.columns if col.startswith('freq_')]]

# Step 3 (optional): Display the results
links_pt_2_l1.head()

,node_id_source,node_id_target,time_distance,stop_name_source,stop_name_target,place_id_source,place_id_target,stop_id_source,stop_id_target,route_id,...,freq_friday_night(dawn),freq_saturday_night(dawn),freq_sunday_night(dawn),freq_week_day_morning,freq_week_day_peak_morning,freq_week_day_afternoon,freq_week_day_peak_evening,freq_week_day_evening,freq_week_day_night,freq_week_day_night(dawn)
0,Bäckängsskolan_307955489_9011014200100000,7:e Villagatan_307955489_9011014200100000,129.0,Bäckängsskolan,7:e Villagatan,307955489,307955489,9022014082676002,9022014082785002,9011014200100000,...,0.0,0.0,0.0,2842.105263,1800.000000,2571.428571,1800.000000,2454.545455,4909.090909,0.0
1,Södra Älvsborgs sjukhus_307955489_901101420010...,7:e Villagatan_307955489_9011014200100000,98.0,Södra Älvsborgs sjukhus,7:e Villagatan,307955489,307955489,9022014082726001,9022014082785001,9011014200100000,...,0.0,0.0,0.0,2781.115880,1825.352113,2623.481781,1800.000000,2347.826087,6000.000000,0.0
2,Bäckängsskolan_307955489_9011014200300000,7:e Villagatan_307955489_9011014200300000,114.0,Bäckängsskolan,7:e Villagatan,307955489,307955489,9022014082676002,9022014082785002,9011014200300000,...,0.0,0.0,0.0,9000.000000,9671.641791,9000.000000,9000.000000,9671.641791,54000.000000,0.0
3,Södra Älvsborgs sjukhus_307955489_901101420030...,7:e Villagatan_307955489_9011014200300000,86.0,Södra Älvsborgs sjukhus,7:e Villagatan,307955489,307955489,9022014082726001,9022014082785001,9011014200300000,...,0.0,0.0,0.0,9000.000000,9671.641791,9000.000000,9000.000000,9671.641791,0.000000,0.0
4,Bäckängsskolan_307955489_9011014200500000,7:e Villagatan_307955489_9011014200500000,117.0,Bäckängsskolan,7:e Villagatan,307955489,307955489,9022014082676002,9022014082785002,9011014200500000,...,0.0,0.0,0.0,8415.584416,5102.362205,8415.584416,4729.927007,6750.000000,9000.000000,0.0


In [12]:
# Create the links of the one direction model
# Step 4: Aggregate Data
links_pt_2_l1['geometry'] = links_pt_2_l1.apply(lambda row: LineString([Point(row['geometry_source']), Point(row['geometry_target'])]), axis=1)
links_pt_2_l1['symmetric_pairs'] = links_pt_2_l1[['node_id_source', 'node_id_target']].apply(lambda x: '-'.join(sorted(x)), axis=1)

# Step 5: Define aggregation functions for various columns
agg_func_link = {
    'route_id': 'first',
    'route_short_name': 'first',
    'mode': 'first',
    'stop_id_source': list,
    'stop_id_target': list,
    'stop_name_source': 'first',
    'stop_name_target': 'first',
    'place_id_source': 'first',
    'place_id_target': 'first',
    'node_id_source': 'first',
    'node_id_target': 'first',
    'direction_id': 'first',
    'geometry': 'first',
    'time_distance': 'mean'
}

freq_cols = [col for col in links_pt_2_l1.columns if col.startswith('freq_')]
agg_func_link.update({col: 'mean' for col in freq_cols})

links_pt_2_l2 = links_pt_2_l1.groupby(['symmetric_pairs']).agg(agg_func_link).reset_index()

# Step 6 (optional): Display the results
links_pt_2_l2.head()

C:\Users\monaliza\AppData\Roaming\Python\Python39\site-packages\geopandas\geodataframe.py:1525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\monaliza\AppData\Roaming\Python\Python39\site-packages\geopandas\geodataframe.py:1525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,symmetric_pairs,route_id,route_short_name,mode,stop_id_source,stop_id_target,stop_name_source,stop_name_target,place_id_source,place_id_target,...,freq_friday_night(dawn),freq_saturday_night(dawn),freq_sunday_night(dawn),freq_week_day_morning,freq_week_day_peak_morning,freq_week_day_afternoon,freq_week_day_peak_evening,freq_week_day_evening,freq_week_day_night,freq_week_day_night(dawn)
0,7:e Villagatan_307955489_9011014200100000-Bäck...,9011014200100000,1,bus_service,"[9022014082676002, 9022014082785001]","[9022014082785002, 9022014082676001]",Bäckängsskolan,7:e Villagatan,307955489,307955489,...,0.0,0.0,0.0,2842.105263,1812.676056,2571.428571,1800.000000,2401.185771,5454.545455,0.0
1,7:e Villagatan_307955489_9011014200100000-Södr...,9011014200100000,1,bus_service,"[9022014082726001, 9022014082785002]","[9022014082785001, 9022014082726008]",Södra Älvsborgs sjukhus,7:e Villagatan,307955489,307955489,...,0.0,0.0,0.0,2769.281344,1830.523082,2634.189870,1800.000000,2369.484999,5454.545455,0.0
2,7:e Villagatan_307955489_9011014200300000-Bäck...,9011014200300000,3,bus_service,"[9022014082676002, 9022014082785001]","[9022014082785002, 9022014082676001]",Bäckängsskolan,7:e Villagatan,307955489,307955489,...,0.0,0.0,0.0,9000.000000,9671.641791,9000.000000,9000.000000,9671.641791,27000.000000,0.0
3,7:e Villagatan_307955489_9011014200300000-Södr...,9011014200300000,3,bus_service,"[9022014082726001, 9022014082785002]","[9022014082785001, 9022014082726008]",Södra Älvsborgs sjukhus,7:e Villagatan,307955489,307955489,...,0.0,0.0,0.0,9000.000000,9671.641791,9000.000000,9000.000000,9671.641791,27000.000000,0.0
4,7:e Villagatan_307955489_9011014200500000-Bäck...,9011014200500000,5,bus_service,"[9022014082676002, 9022014082785001]","[9022014082785002, 9022014082676001]",Bäckängsskolan,7:e Villagatan,307955489,307955489,...,0.0,0.0,0.0,8064.935065,4801.181102,8064.935065,4614.963504,7476.265823,9000.000000,0.0


In [13]:
# Filter and organize the final table
# Step 7: Rename columns for clarity
links_pt_2_l2.rename(
    columns={'node_id_source': 'source', 'node_id_target': 'target'},
    inplace=True
)

# Step 8: Select desired columns for the final DataFrame
links_pt_2 = links_pt_2_l2[['source', 'target', 'time_distance', 'route_id', 'stop_name_source',
                            'place_id_source', 'stop_name_target',
                            'place_id_target', 'stop_id_source', 'stop_id_target',
                            'route_short_name', 'mode', 'direction_id',
                            'geometry'] + [col for col in links_pt_2_l2.columns if col.startswith('freq_')]]

# Step 9: Filter out rows where 'source' is equal to 'target'
links_pt_2 = links_pt_2[links_pt_2['source'] != links_pt_2['target']]

# Step 10 (optional): Display the results
links_pt_2.head()

,source,target,time_distance,route_id,stop_name_source,place_id_source,stop_name_target,place_id_target,stop_id_source,stop_id_target,...,freq_friday_night(dawn),freq_saturday_night(dawn),freq_sunday_night(dawn),freq_week_day_morning,freq_week_day_peak_morning,freq_week_day_afternoon,freq_week_day_peak_evening,freq_week_day_evening,freq_week_day_night,freq_week_day_night(dawn)
0,Bäckängsskolan_307955489_9011014200100000,7:e Villagatan_307955489_9011014200100000,128.5,9011014200100000,Bäckängsskolan,307955489,7:e Villagatan,307955489,"[9022014082676002, 9022014082785001]","[9022014082785002, 9022014082676001]",...,0.0,0.0,0.0,2842.105263,1812.676056,2571.428571,1800.000000,2401.185771,5454.545455,0.0
1,Södra Älvsborgs sjukhus_307955489_901101420010...,7:e Villagatan_307955489_9011014200100000,117.5,9011014200100000,Södra Älvsborgs sjukhus,307955489,7:e Villagatan,307955489,"[9022014082726001, 9022014082785002]","[9022014082785001, 9022014082726008]",...,0.0,0.0,0.0,2769.281344,1830.523082,2634.189870,1800.000000,2369.484999,5454.545455,0.0
2,Bäckängsskolan_307955489_9011014200300000,7:e Villagatan_307955489_9011014200300000,112.0,9011014200300000,Bäckängsskolan,307955489,7:e Villagatan,307955489,"[9022014082676002, 9022014082785001]","[9022014082785002, 9022014082676001]",...,0.0,0.0,0.0,9000.000000,9671.641791,9000.000000,9000.000000,9671.641791,27000.000000,0.0
3,Södra Älvsborgs sjukhus_307955489_901101420030...,7:e Villagatan_307955489_9011014200300000,103.0,9011014200300000,Södra Älvsborgs sjukhus,307955489,7:e Villagatan,307955489,"[9022014082726001, 9022014082785002]","[9022014082785001, 9022014082726008]",...,0.0,0.0,0.0,9000.000000,9671.641791,9000.000000,9000.000000,9671.641791,27000.000000,0.0
4,Bäckängsskolan_307955489_9011014200500000,7:e Villagatan_307955489_9011014200500000,123.5,9011014200500000,Bäckängsskolan,307955489,7:e Villagatan,307955489,"[9022014082676002, 9022014082785001]","[9022014082785002, 9022014082676001]",...,0.0,0.0,0.0,8064.935065,4801.181102,8064.935065,4614.963504,7476.265823,9000.000000,0.0


In [16]:
# Step 11 (optional): create the GeoDataFrame and export to the DB
links_pt_2 = gpd.GeoDataFrame(links_pt_2, geometry='geometry', crs='3006')
links_pt_2.to_postgis('links_pt_2', engine, schema='pt_2_odirection_mstops_mroutes', if_exists ='replace')

### Transfers

#### Same stop

In [17]:
# Preprocess the Data
# Step 1: Create a copy of the nodes table
nodes_transfer_same_stop = nodes_pt_2[['node_id', 'stop_name', 'place_id', 'geometry']].copy()

#Step 2 (optional): display the results
nodes_transfer_same_stop.head()

,node_id,stop_name,place_id,geometry
0,7:e Villagatan_307955489_9011014200100000,7:e Villagatan,307955489,POINT (378291.977 6399668.440)
1,7:e Villagatan_307955489_9011014200300000,7:e Villagatan,307955489,POINT (378291.977 6399668.440)
2,7:e Villagatan_307955489_9011014200500000,7:e Villagatan,307955489,POINT (378291.977 6399668.440)
3,7:e Villagatan_307955489_9011014200800000,7:e Villagatan,307955489,POINT (378291.977 6399668.440)
4,AGA Vattenfall_307980814_9011014633100000,AGA Vattenfall,307980814,POINT (313191.955 6442732.480)


In [18]:
# Step 3: Merge the stops in order to create the transfers
# Source
stop_routes_source = nodes_transfer_same_stop.rename(columns={'node_id': 'source', 'stop_name': 'target'}).drop_duplicates()

# Target
stop_routes_target = pd.merge(nodes_transfer_same_stop,
                              nodes_pt_2[['node_id']],
                              on='node_id',
                              how='inner').rename(columns={'stop_name': 'source', 'node_id': 'target'}).drop_duplicates()

# Step 4: Create the transfers same stop table
transfers_same_stop = pd.merge(stop_routes_source, stop_routes_target, how='outer')

#Step 5 (optional): Display the results
transfers_same_stop.head()

,source,target,place_id,geometry
0,7:e Villagatan,7:e Villagatan_307955489_9011014200100000,307955489,POINT (378291.977 6399668.440)
1,7:e Villagatan,7:e Villagatan_307955489_9011014200300000,307955489,POINT (378291.977 6399668.440)
2,7:e Villagatan,7:e Villagatan_307955489_9011014200500000,307955489,POINT (378291.977 6399668.440)
3,7:e Villagatan,7:e Villagatan_307955489_9011014200800000,307955489,POINT (378291.977 6399668.440)
4,7:e Villagatan_307955489_9011014200100000,7:e Villagatan,307955489,POINT (378291.977 6399668.440)


In [19]:
# Step 6: Merge the table with the frequency of the links table
transfers_same_stop = pd.merge(transfers_same_stop,
                               links_pt_2[['target'] + [col for col in links_pt_2.columns if col.startswith('freq_')]],
                               on='target',
                               how='outer').drop_duplicates(subset=('source', 'target'))
transfers_same_stop = transfers_same_stop[['source', 'target', 'geometry'] + [col for col in transfers_same_stop.columns if col.startswith('freq_')]]

#Step 7 (optional): Display the results
transfers_same_stop.head()

,source,target,geometry,freq_monday_morning,freq_tuesday_morning,freq_wednesday_morning,freq_thursday_morning,freq_friday_morning,freq_saturday_morning,freq_sunday_morning,...,freq_friday_night(dawn),freq_saturday_night(dawn),freq_sunday_night(dawn),freq_week_day_morning,freq_week_day_peak_morning,freq_week_day_afternoon,freq_week_day_peak_evening,freq_week_day_evening,freq_week_day_night,freq_week_day_night(dawn)
0,7:e Villagatan_307955489_9011014200100000,7:e Villagatan,POINT (378291.977 6399668.440),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7:e Villagatan_307955489_9011014200300000,7:e Villagatan,POINT (378291.977 6399668.440),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7:e Villagatan_307955489_9011014200500000,7:e Villagatan,POINT (378291.977 6399668.440),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7:e Villagatan_307955489_9011014200800000,7:e Villagatan,POINT (378291.977 6399668.440),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7:e Villagatan,7:e Villagatan_307955489_9011014200100000,POINT (378291.977 6399668.440),568.421053,568.421053,568.421053,568.421053,568.421053,646.915352,647.778063,...,0.0,0.0,0.0,2842.105263,1812.676056,2571.428571,1800.0,2401.185771,5454.545455,0.0


In [20]:
# Step 8: Replace 0 with NaN and vice versa in frequency columns
freq_columns = [col for col in transfers_same_stop.columns if col.startswith('freq_')]
transfers_same_stop[freq_columns] = transfers_same_stop[freq_columns].replace({0: np.nan, pd.NA: 0})

#Step 9 (optional): Display the results
transfers_same_stop.head()

,source,target,geometry,freq_monday_morning,freq_tuesday_morning,freq_wednesday_morning,freq_thursday_morning,freq_friday_morning,freq_saturday_morning,freq_sunday_morning,...,freq_friday_night(dawn),freq_saturday_night(dawn),freq_sunday_night(dawn),freq_week_day_morning,freq_week_day_peak_morning,freq_week_day_afternoon,freq_week_day_peak_evening,freq_week_day_evening,freq_week_day_night,freq_week_day_night(dawn)
0,7:e Villagatan_307955489_9011014200100000,7:e Villagatan,POINT (378291.977 6399668.440),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7:e Villagatan_307955489_9011014200300000,7:e Villagatan,POINT (378291.977 6399668.440),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7:e Villagatan_307955489_9011014200500000,7:e Villagatan,POINT (378291.977 6399668.440),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7:e Villagatan_307955489_9011014200800000,7:e Villagatan,POINT (378291.977 6399668.440),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7:e Villagatan,7:e Villagatan_307955489_9011014200100000,POINT (378291.977 6399668.440),568.421053,568.421053,568.421053,568.421053,568.421053,646.915352,647.778063,...,NaN,NaN,NaN,2842.105263,1812.676056,2571.428571,1800.0,2401.185771,5454.545455,NaN


In [21]:
# Step 10 (optional): Set geometry and export to the DB
transfers_same_stop = gpd.GeoDataFrame(transfers_same_stop, geometry='geometry', crs='3006')
transfers_same_stop.to_postgis('transfers_same_stop_pt_2', engine, schema='pt_2_odirection_mstops_mroutes', if_exists ='replace')

#### Different stops

In [22]:
#Preprocess the Data
# Step 1: Create a copy of the nodes table
nodes_transfer_diff_stop = nodes_pt_2[['node_id', 'stop_name', 'place_id', 'geometry']].copy()

# Step 2 (optional): Display the results
nodes_transfer_diff_stop.head()

,node_id,stop_name,place_id,geometry
0,7:e Villagatan_307955489_9011014200100000,7:e Villagatan,307955489,POINT (378291.977 6399668.440)
1,7:e Villagatan_307955489_9011014200300000,7:e Villagatan,307955489,POINT (378291.977 6399668.440)
2,7:e Villagatan_307955489_9011014200500000,7:e Villagatan,307955489,POINT (378291.977 6399668.440)
3,7:e Villagatan_307955489_9011014200800000,7:e Villagatan,307955489,POINT (378291.977 6399668.440)
4,AGA Vattenfall_307980814_9011014633100000,AGA Vattenfall,307980814,POINT (313191.955 6442732.480)


In [23]:
# Create the buffers
# Step 3: Create a buffer column and set it as geometry
nodes_transfer_diff_stop['buffer'] = nodes_transfer_diff_stop['geometry'].buffer(85)
nodes_transfer_diff_stop = nodes_transfer_diff_stop.set_geometry('buffer',
                                                                 crs='EPSG:3006')

# Step 4: found the intersections among the buffers
nodes_transfer_diff_stop = gpd.overlay(nodes_transfer_diff_stop,
                                       nodes_transfer_diff_stop,
                                       how='intersection').rename(columns={'node_id_1':'source',
                                                                           'node_id_2':'target',
                                                                           'stop_name_1':'stop_name_source',
                                                                           'stop_name_2':'stop_name_target',
                                                                           'geometry_1': 'geometry_source',
                                                                           'geometry_2': 'geometry_target'}).dropna(subset=['source'])
# Step 5 (optional): Display the results
nodes_transfer_diff_stop.head()

,source,stop_name_source,place_id_1,geometry_source,target,stop_name_target,place_id_2,geometry_target,geometry
0,7:e Villagatan_307955489_9011014200100000,7:e Villagatan,307955489,POINT (378291.977 6399668.440),7:e Villagatan_307955489_9011014200100000,7:e Villagatan,307955489,POINT (378291.977 6399668.440),"POLYGON ((378376.568 6399660.109, 378375.344 6..."
1,7:e Villagatan_307955489_9011014200100000,7:e Villagatan,307955489,POINT (378291.977 6399668.440),7:e Villagatan_307955489_9011014200300000,7:e Villagatan,307955489,POINT (378291.977 6399668.440),"POLYGON ((378376.568 6399660.109, 378375.344 6..."
2,7:e Villagatan_307955489_9011014200100000,7:e Villagatan,307955489,POINT (378291.977 6399668.440),7:e Villagatan_307955489_9011014200500000,7:e Villagatan,307955489,POINT (378291.977 6399668.440),"POLYGON ((378376.568 6399660.109, 378375.344 6..."
3,7:e Villagatan_307955489_9011014200100000,7:e Villagatan,307955489,POINT (378291.977 6399668.440),7:e Villagatan_307955489_9011014200800000,7:e Villagatan,307955489,POINT (378291.977 6399668.440),"POLYGON ((378376.568 6399660.109, 378375.344 6..."
4,7:e Villagatan_307955489_9011014200300000,7:e Villagatan,307955489,POINT (378291.977 6399668.440),7:e Villagatan_307955489_9011014200100000,7:e Villagatan,307955489,POINT (378291.977 6399668.440),"POLYGON ((378376.568 6399660.109, 378375.344 6..."


In [24]:
#Create the geometry
# Step 6: Create linstrings between the intersections
nodes_transfer_diff_stop['geometry'] = nodes_transfer_diff_stop.apply(
    lambda row: LineString([[row.geometry_source.x,  row.geometry_source.y],
                            [row.geometry_target.x,row.geometry_target.y]])
    if row.geometry_target is not None else None,
    axis=1
)

# Step 7: Set the length of the linestring
nodes_transfer_diff_stop['length'] = nodes_transfer_diff_stop['geometry'].length
nodes_transfer_diff_stop['time_distance'] = nodes_transfer_diff_stop['length'] / 1.2

# Step 8 (optional): Display the results
nodes_transfer_diff_stop.head()

,source,stop_name_source,place_id_1,geometry_source,target,stop_name_target,place_id_2,geometry_target,geometry,length,time_distance
0,7:e Villagatan_307955489_9011014200100000,7:e Villagatan,307955489,POINT (378291.977 6399668.440),7:e Villagatan_307955489_9011014200100000,7:e Villagatan,307955489,POINT (378291.977 6399668.440),"LINESTRING (378291.977 6399668.440, 378291.977...",0.0,0.0
1,7:e Villagatan_307955489_9011014200100000,7:e Villagatan,307955489,POINT (378291.977 6399668.440),7:e Villagatan_307955489_9011014200300000,7:e Villagatan,307955489,POINT (378291.977 6399668.440),"LINESTRING (378291.977 6399668.440, 378291.977...",0.0,0.0
2,7:e Villagatan_307955489_9011014200100000,7:e Villagatan,307955489,POINT (378291.977 6399668.440),7:e Villagatan_307955489_9011014200500000,7:e Villagatan,307955489,POINT (378291.977 6399668.440),"LINESTRING (378291.977 6399668.440, 378291.977...",0.0,0.0
3,7:e Villagatan_307955489_9011014200100000,7:e Villagatan,307955489,POINT (378291.977 6399668.440),7:e Villagatan_307955489_9011014200800000,7:e Villagatan,307955489,POINT (378291.977 6399668.440),"LINESTRING (378291.977 6399668.440, 378291.977...",0.0,0.0
4,7:e Villagatan_307955489_9011014200300000,7:e Villagatan,307955489,POINT (378291.977 6399668.440),7:e Villagatan_307955489_9011014200100000,7:e Villagatan,307955489,POINT (378291.977 6399668.440),"LINESTRING (378291.977 6399668.440, 378291.977...",0.0,0.0


In [25]:
#Filter and organize the tables
# Step 9: Filter the transfers between different stops
transfers_diff_stop = nodes_transfer_diff_stop[nodes_transfer_diff_stop['length'] > 0]

# Step 10: Drop the duplicates of symmetric pairs
transfers_diff_stop ['symmetric_pairs'] = transfers_diff_stop[['source',
                                                               'target']].apply(lambda x: '-'.join(sorted(x)),
                                                                                axis=1)
transfers_diff_stop = transfers_diff_stop.drop_duplicates(subset = ['symmetric_pairs'])

# Step 11: Filter the necessary columns
transfers_diff_stop = transfers_diff_stop [['stop_name_source',
                                            'stop_name_target',
                                            'geometry',
                                            'length',
                                            'time_distance']].rename(columns={'stop_name_source':'source',
                                                                              'stop_name_target':'target'}).drop_duplicates()
# Step 12 (optional): Display the results
transfers_diff_stop.head()

C:\Users\monaliza\AppData\Roaming\Python\Python39\site-packages\geopandas\geodataframe.py:1525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,source,target,geometry,length,time_distance
19,AGA Vattenfall,AGA Vattenfall,"LINESTRING (313191.955 6442732.480, 313179.952...",37.481900,31.234917
33,Abrahamstorp,Abrahamstorp,"LINESTRING (371017.481 6448504.455, 371000.976...",32.936222,27.446852
40,Acklinga kyrka,Acklinga kyrka,"LINESTRING (434071.965 6451013.922, 434077.950...",6.070173,5.058478
49,Acklinga,Acklinga,"LINESTRING (434427.979 6451013.415, 434427.976...",5.512155,4.593462
66,Agerhus,Jordbron,"LINESTRING (324621.469 6533086.938, 324604.463...",125.603619,104.669683


In [26]:
# Step 10 (optional): Set geometry and export to the DB
transfers_diff_stop= gpd.GeoDataFrame(transfers_diff_stop, geometry='geometry', crs='3006')
transfers_diff_stop.to_postgis('transfers_diff_stop_pt_2', engine, schema='pt_2_odirection_mstops_mroutes', if_exists ='replace')